In [5]:
import time
import open3d as o3d
import numpy as np
from m2dp import M2DP

pcd = o3d.io.read_point_cloud("data/bunny/data/bun000.ply")
print(pcd)
print(np.asarray(pcd.points))
#o3d.visualization.draw_geometries([pcd])


PointCloud with 40256 points.
[[-0.06325    0.0359793  0.0420873]
 [-0.06275    0.0360343  0.0425949]
 [-0.0645     0.0365101  0.0404362]
 ...
 [-0.01575    0.187201  -0.0220209]
 [-0.01525    0.187218  -0.0237782]
 [-0.018      0.18794   -0.0197253]]


In [6]:
def dist(x,y):
    '''
        ||euklidean distance ^2||
    '''
    x = np.asarray(x)
    y = np.asarray(y)
    return np.linalg.norm((x-y)*(x-y))

def M2DP_desc(seg):
    """
        generate pointcloud signature
    """
    des, A1 = M2DP(seg)
    return des

def M2DP_iss_desc(seg):
    """
        generate pointcloud signature from 
        iss simplified pointcloud
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(seg)
    keypoints = o3d.geometry.keypoint.compute_iss_keypoints(pcd)
    
    des, A1 = M2DP(keypoints.points)
    return des

def M2DP_downsample_desc(seg, voxel_size):
    """
        pointcloud signature downsampled to 
        VOXEL_SIZEd voxels
    """
    VOXEL_SIZE = voxel_size
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(seg)
    downpcd = pcd.voxel_down_sample(voxel_size=VOXEL_SIZE)
    size = len(downpcd.points)
    des, A1 = M2DP(downpcd.points)
    return des, size
    
    

## M2DP with downsampling comparison

In [7]:
t1 = time.time()
sign1 = M2DP_desc(pcd.points)
t2 = time.time()
t = t2 - t1

# t1 = time.time()
# sign2 = M2DP_iss_desc(pcd.points)
# t2 = time.time()
# t = t2 - t1
# print("M2DP iss comp time:", t)

t1 = time.time()
signtv1, size1 = M2DP_downsample_desc(pcd.points, 0.01)
t2 = time.time()
tv1 = t2 - t1

t1 = time.time()
signtv2, size2 = M2DP_downsample_desc(pcd.points, 0.02)
t2 = time.time()
tv2 = t2 - t1

t1 = time.time()
signtv5, size5 = M2DP_downsample_desc(pcd.points, 0.05)
t2 = time.time()
tv5 = t2 - t1

#print("d(iss, orig) = ", dist(sign1, sign2))
print("standard M2DP t =",t, "d(bunnyorig, bunnyorig) =", dist(sign1, sign1), len(pcd.points), "points")
print("vox 0.01 t =",tv1, "d(down _0.01, bunnyorig) =", dist(sign1, signtv1), size1, "points")
print("vox 0.02 t =",tv2, "d(down_0.02, bunnyorig) =", dist(sign1, signtv2), size2, "points")
print("vox 0.05 t =",tv5, "d(down_0.05, bunnyorig) =", dist(sign1, signtv5), size5, "points")

standard M2DP t = 0.5277140140533447 d(bunnyorig, bunnyorig) = 0.0 40256 points
vox 0.01 t = 0.0357060432434082 d(down _0.01, bunnyorig) = 0.05542126742068622 376 points
vox 0.02 t = 0.03385281562805176 d(down_0.02, bunnyorig) = 0.06428948236640031 113 points
vox 0.05 t = 0.031661033630371094 d(down_0.05, bunnyorig) = 0.23587941025180628 23 points


In [8]:
# compare to dragon
dragon_pcd = o3d.io.read_point_cloud("data/dragon/dragonStandRight_0.ply")
print(dragon_pcd)

t1 = time.time()
signdrag1 = M2DP_desc(dragon_pcd.points)
t2 = time.time()
td0 = t2 - t1

t1 = time.time()
signdrag01, ld01 = M2DP_downsample_desc(dragon_pcd.points, 0.01)
t2 = time.time()
td01 = t2 - t1

t1 = time.time()
signdrag02, ld02 = M2DP_downsample_desc(dragon_pcd.points, 0.02)
t2 = time.time()
td02 = t2 - t1

t1 = time.time()
signdrag05, ld05 = M2DP_downsample_desc(dragon_pcd.points, 0.05)
t2 = time.time()
td05 = t2 - t1

print("dragon t =",td0,  "d(dragon, bunnyorig) =", dist(signdrag1, sign1), len(dragon_pcd.points), "points")
print("dragon t =",td01, "d(dragon, bunnyorig) =", dist(signdrag01, signtv1), ld01, "points")
print("dragon t =",td02, "d(dragon, bunnyorig) =", dist(signdrag02, signtv2), ld02, "points")
print("dragon t =",td05, "d(dragon, bunnyorig) =", dist(signdrag05, signtv5), ld05, "points")

PointCloud with 41841 points.
dragon t = 0.4260294437408447 d(dragon, bunnyorig) = 0.07035753104401343 41841 points
dragon t = 0.03274250030517578 d(dragon, bunnyorig) = 0.08631220694357869 461 points
dragon t = 0.029366493225097656 d(dragon, bunnyorig) = 0.10555629296329874 135 points
dragon t = 0.02659296989440918 d(dragon, bunnyorig) = 0.13614170866036135 29 points


In [9]:
# classify different bunny segment and dragon

bunny2_pcd = o3d.io.read_point_cloud("data/bunny/data/bun045.ply")


bunny2_1 = M2DP_desc(bunny2_pcd.points)

bunny2_01, ld01 = M2DP_downsample_desc(bunny2_pcd.points, 0.01)

bunny2_02, ld02 = M2DP_downsample_desc(bunny2_pcd.points, 0.02)

bunny2_05, ld05 = M2DP_downsample_desc(bunny2_pcd.points, 0.05)

print("d(bunny2, bunnyorig) =", dist(bunny2_1, sign1), len(bunny2_pcd.points), "points")
print("d(bunny2, bunnyorig) =", dist(bunny2_01, signtv1), ld01, "points")
print("d(bunny2, bunnyorig) =", dist(bunny2_02, signtv2), ld02, "points")
print("d(bunny2, bunnyorig) =", dist(bunny2_05, signtv5), ld05, "points")

d(bunny2, bunnyorig) = 0.010066959670916953 40097 points
d(bunny2, bunnyorig) = 0.00898483412864675 371 points
d(bunny2, bunnyorig) = 0.06474951499101353 114 points
d(bunny2, bunnyorig) = 0.15107202873535808 28 points


In [10]:
# visualisations

# This function is only used to make the keypoints look better on the rendering
def keypoints_to_spheres(keypoints):
    spheres = o3d.geometry.TriangleMesh()
    for keypoint in keypoints.points:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.001)
        sphere.translate(keypoint)
        spheres += sphere
    spheres.paint_uniform_color([0.1, 0.1, 0.8])
    return spheres


#original
o3d.visualization.draw_geometries([pcd])
#iss
keypoints = o3d.geometry.keypoint.compute_iss_keypoints(pcd)
o3d.visualization.draw_geometries([keypoints_to_spheres(keypoints)])
#downsampled
downpcd = pcd.voxel_down_sample(voxel_size=0.02)
o3d.visualization.draw_geometries([keypoints_to_spheres(downpcd)])